This tries alternate ways of fitting w_pen

In [11]:
import pickle
import random
import datetime

import scipy
import numpy as np
import pandas as pd

try:
    import SparseSC as SC
except ImportError:
    raise RuntimeError("SparseSC is not installed. Use 'pip install -e .' or 'conda develop .' from repo root to install in dev mode")

In [12]:
random.seed(12345)
np.random.seed(101101001)
n_multi=5

In [13]:
smoking_df = pd.read_stata("../replication/smoking.dta")
smoking_df['year'] = smoking_df['year'].astype('int')
smoking_df = smoking_df.set_index(['state', 'year']).sort_index()
smoking_df.head()

cigsale  lnincome  beer  age15to24   retprice
state   year                                                  
Alabama 1970   89.800003       NaN   NaN   0.178862  39.599998
        1971   95.400002       NaN   NaN   0.179928  42.700001
        1972  101.099998  9.498476   NaN   0.180994  42.299999
        1973  102.900002  9.550107   NaN   0.182060  42.099998
        1974  108.199997  9.537163   NaN   0.183126  43.099998

In [14]:
Y = smoking_df[['cigsale']].unstack('year')
Y_cols = Y.columns
Y.head()

cigsale                                                  \
year               1970        1971        1972        1973        1974   
state                                                                     
Alabama       89.800003   95.400002  101.099998  102.900002  108.199997   
Arkansas     100.300003  104.099998  103.900002  108.000000  109.699997   
California   123.000000  121.000000  123.500000  124.400002  126.699997   
Colorado     124.800003  125.500000  134.300003  137.899994  132.800003   
Connecticut  120.000000  117.599998  110.800003  109.300003  112.400002   

                                                                         ...  \
year               1975        1976        1977        1978        1979  ...   
state                                                                    ...   
Alabama      111.699997  116.199997  117.099998  123.000000  121.400002  ...   
Arkansas     114.800003  119.099998  122.599998  127.300003  126.500000  ...   
California   127.099998  128.000000  126.400002  126.099998  121.900002  ...   
Colorado     131.000000  134.199997  132.000000  129.199997  131.500000  ...   
Connecticut  110.199997  113.400002  117.300003  117.500000  117.400002  ...   

                                                                         \
year               1991        1992        1993        1994        1995   
state                                                                     
Alabama      107.900002  109.099998  108.500000  107.099998  102.599998   
Arkansas     116.800003  126.000000  113.800003  108.800003  113.000000   
California    68.699997   67.500000   63.400002   58.599998   56.400002   
Colorado      90.199997   88.300003   88.599998   89.099998   85.400002   
Connecticut   86.699997   83.500000   79.099998   76.599998   79.300003   

                                                                        
year               1996        1997        1998        1999       2000  
state                                                                   
Alabama      101.400002  104.900002  106.199997  100.699997  96.199997  
Arkansas     110.699997  108.699997  109.500000  104.800003  99.400002  
California    54.500000   53.799999   52.299999   47.200001  41.599998  
Colorado      83.099998   81.300003   81.199997   79.599998  73.000000  
Connecticut   76.000000   75.900002   75.500000   73.400002  71.400002  

[5 rows x 31 columns]

In [15]:
T0 = 19
i_t = 2 #unit 3, but zero-index
treated_units = [i_t]
control_units = [u for u in range(Y.shape[0]) if u not in treated_units]
print(Y.shape)
print(control_units)

(39, 31)
[0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38]


In [16]:
Y_names = Y.columns.get_level_values('year')
Y_pre_names = ["cigsale(" + str(i) + ")" for i in Y_names[:T0]]
print(Y.isnull().sum().sum()) #0
Y = Y.values
T = Y.shape[1]
T1 = T-T0
Y_pre,Y_post = Y[:,:T0], Y[:,T0:]

0


In [17]:
# Stata: synth cigsale beer(1984(1)1988) lnincome retprice age15to24 cigsale(1988) cigsale(1980) cigsale(1975), xperiod(1980(1)1988)  trunit(3) trperiod(1989) 

year_ind = smoking_df.index.get_level_values('year')
beer_pre = smoking_df.loc[np.logical_and(year_ind>=1984, year_ind<=1988),["beer"]]
Xother_pre = smoking_df.loc[np.logical_and(year_ind>=1980, year_ind<=1988), ['lnincome', 'retprice', 'age15to24']]
X_avgs = pd.concat((beer_pre.groupby('state').mean(), 
                    Xother_pre.groupby('state').mean())
                   , axis=1)

In [18]:
X_full = pd.concat((X_avgs, beer_pre.unstack('year'), Xother_pre.unstack('year')), axis=1)
X_full_names = [c[0] + "(" + str(c[1]) + ")" if len(c)==2 else c for c in X_full.columns]
X_full.isnull().sum().sum() #0
X_full = X_full.values
X_Y_pre = np.concatenate((X_full, Y_pre), axis=1)
X_Y_pre_names = X_full_names + Y_pre_names
X_Y_pre_names_arr = np.array(X_Y_pre_names)

In [19]:
def print_summary(fit, Y_pre, Y_post, Y_sc, fast=False, show_noNH = False):
    Y_pre_sc,Y_post_sc = Y_sc[:,:T0], Y_sc[:,T0:]
    V_vec = fit.match_space_desc if fast else np.diag(fit.V)
    #print("V: " + str(V_vec))
    print("V>0: " + str(V_vec[V_vec>0]))
    print("#V>0: " + str(sum(V_vec[V_vec>0])))
    Y_pre_effect_c = Y_pre[control_units, :] - Y_pre_sc[control_units, :]
    Y_post_effect_c = Y_post[control_units, :] - Y_post_sc[control_units, :]
    
    print(X_Y_pre_names_arr[V_vec>0])

    def print_seg_info(arr, seg_name):
        print("Avg bias " + seg_name + ": " + str(arr.mean()))
        print(scipy.stats.ttest_1samp(arr.flatten(), popmean=0)) 
        print("Avg MSE " + seg_name + ": " + str(np.mean(np.power(arr, 2))) )
        print("Avg max abs val " + seg_name + ":" + str(np.mean(np.amax(np.abs(arr), axis=0))))
    
    print_seg_info(Y_pre_effect_c, "pre")
    print_seg_info(Y_post_effect_c, "post")

    NH_idx = 20 #1-based index including treatment is 22
    if show_noNH:    
        Y_pre_effect_c_noNH = np.delete(Y_pre_effect_c, NH_idx, axis=0)
        Y_post_effect_c_noNH = np.delete(Y_post_effect_c, NH_idx, axis=0)    
        
        print_seg_info(Y_pre_effect_c_noNH, "pre (no-NH)")
        print_seg_info(Y_post_effect_c_noNH, "post (no-NH)")


# Fast

In [20]:
print("Initial fit")
fast_fit = SC.fit_fast(X_Y_pre, Y_post, treated_units=[i_t], w_pen_inner=False)
fast_Y_sc = fast_fit.predict(Y)
print_summary(fast_fit, Y_pre, Y_post, fast_Y_sc, fast=True)

print("Cross fit")
print("Start time: {}".format(datetime.datetime.now().replace(microsecond=0)))
honest_predictions_fast, cf_fits_fast = SC.get_c_predictions_honest(X_Y_pre[control_units,:], Y_post[control_units,:], Y[control_units,:], match_space_maker=SC.MTLasso_MatchSpace_factory(v_pen=fast_fit.fitted_v_pen), cf_folds=38, verbose=1, progress=False, fast=True, n_multi=0, w_pen_inner=False)
print("End time: {}".format(datetime.datetime.now().replace(microsecond=0)))
fast_Y_sc_xf = fast_Y_sc.copy()
fast_Y_sc_xf[control_units,:] = honest_predictions_fast
print_summary(fast_fit, Y_pre, Y_post, fast_Y_sc_xf, fast=True)

print("Cross fit2")
print("Start time: {}".format(datetime.datetime.now().replace(microsecond=0)))
honest_predictions_fast2, cf_fits_fast2 = SC.get_c_predictions_honest(X_Y_pre[control_units,:], Y_post[control_units,:], Y[control_units,:], cf_folds=38, verbose=1, progress=False, fast=True, n_multi=n_multi, w_pen_inner=False)
print("End time: {}".format(datetime.datetime.now().replace(microsecond=0)))
fast_Y_sc_xf2 = fast_Y_sc.copy()
fast_Y_sc_xf2[control_units,:] = honest_predictions_fast2
print_summary(fast_fit, Y_pre, Y_post, fast_Y_sc_xf2, fast=True)

Initial fit
V>0: [  0.51300713  22.96821089  66.11994103  20.37246761   0.39536433
 146.42793943   0.4007635    1.09141769   1.6124491 ]
#V>0: 259.9015607131534
['beer(1986)' 'lnincome(1985)' 'lnincome(1987)' 'lnincome(1988)'
 'retprice(1985)' 'age15to24(1988)' 'cigsale(1986)' 'cigsale(1987)'
 'cigsale(1988)']
Avg bias pre: 0.4301708231606254
Ttest_1sampResult(statistic=0.771940885975491, pvalue=0.44040237737110044)
Avg MSE pre: 224.0824610330939
Avg max abs val pre:43.172209630811594
Avg bias post: -0.08712370978264666
Ttest_1sampResult(statistic=-0.17214648490759957, pvalue=0.8633989092834)
Avg MSE post: 116.55102105857189
Avg max abs val post:26.988419004981978
Cross fit
Start time: 2021-01-21 14:11:06
CROSS-FITTING:
 |>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>| 
End time: 2021-01-21 14:11:19
V>0: [  0.51300713  22.96821089  66.11994103  20.37246761   0.39536433
 146.42793943   0.4007635    1.09141769   1.6124491 ]
#V>0: 259.

# Full

In [21]:
#Try with the simplex constraints
print("Start time: {}".format(datetime.datetime.now().replace(microsecond=0)))
full_simplex_fit = SC.fit(X_Y_pre, Y_post, treated_units=[i_t], verbose=0, progress=False, print_path=False, constrain="simplex", w_pen_inner=True)
print("End time: {}".format(datetime.datetime.now().replace(microsecond=0)))
full_simplex_Y_sc = full_simplex_fit.predict(Y)
print_summary(full_simplex_fit, Y_pre, Y_post, full_simplex_Y_sc)

Start time: 2021-01-21 14:12:54
End time: 2021-01-21 14:17:43
V>0: [0.04306114 0.04854367 0.05662882 0.05987692 0.0607689  0.06293688
 0.0692605  0.06679674 0.06189289 0.05663904 0.05521077 0.05218668
 0.05268511 0.04874349 0.04189329 0.04094062 0.0411164  0.03987489
 0.04094325]
#V>0: 1.0
['cigsale(1970)' 'cigsale(1971)' 'cigsale(1972)' 'cigsale(1973)'
 'cigsale(1974)' 'cigsale(1975)' 'cigsale(1976)' 'cigsale(1977)'
 'cigsale(1978)' 'cigsale(1979)' 'cigsale(1980)' 'cigsale(1981)'
 'cigsale(1982)' 'cigsale(1983)' 'cigsale(1984)' 'cigsale(1985)'
 'cigsale(1986)' 'cigsale(1987)' 'cigsale(1988)']
Avg bias pre: -0.0011602726698260735
Ttest_1sampResult(statistic=-0.02574678064478845, pvalue=0.9794664348511534)
Avg MSE pre: 1.4642320989368507
Avg max abs val pre:4.625741609061293
Avg bias post: -0.2166529637721898
Ttest_1sampResult(statistic=-0.37873110147816, pvalue=0.7050642121675874)
Avg MSE post: 148.94146183684256
Avg max abs val post:33.15973068544222


In [22]:
# Cross-fitting keeping the full-fit penalty parameters
print("Start time: {}".format(datetime.datetime.now().replace(microsecond=0)))
honest_predictions_simplex, cf_fits_full_simplex = SC.get_c_predictions_honest(X_Y_pre[control_units,:], Y_post[control_units,:], Y[control_units,:], w_pen=full_simplex_fit.fitted_w_pen, v_pen=full_simplex_fit.fitted_v_pen, cf_folds=38, verbose=1, progress=False, fast=False, n_multi=n_multi, constrain="simplex", w_pen_inner=True)
print("End time: {}".format(datetime.datetime.now().replace(microsecond=0)))
full_simplex_Y_sc_xf = full_simplex_Y_sc.copy()
full_simplex_Y_sc_xf[control_units,:] = honest_predictions_simplex
print_summary(full_simplex_fit, Y_pre, Y_post, full_simplex_Y_sc_xf)

Start time: 2021-01-21 14:17:43
CROSS-FITTING:
 |>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>| 
End time: 2021-01-21 14:18:06
V>0: [0.04306114 0.04854367 0.05662882 0.05987692 0.0607689  0.06293688
 0.0692605  0.06679674 0.06189289 0.05663904 0.05521077 0.05218668
 0.05268511 0.04874349 0.04189329 0.04094062 0.0411164  0.03987489
 0.04094325]
#V>0: 1.0
['cigsale(1970)' 'cigsale(1971)' 'cigsale(1972)' 'cigsale(1973)'
 'cigsale(1974)' 'cigsale(1975)' 'cigsale(1976)' 'cigsale(1977)'
 'cigsale(1978)' 'cigsale(1979)' 'cigsale(1980)' 'cigsale(1981)'
 'cigsale(1982)' 'cigsale(1983)' 'cigsale(1984)' 'cigsale(1985)'
 'cigsale(1986)' 'cigsale(1987)' 'cigsale(1988)']
Avg bias pre: 0.0015822540029594443
Ttest_1sampResult(statistic=0.032882573687673514, pvalue=0.9737773300560144)
Avg MSE pre: 1.6693858119951455
Avg max abs val pre:4.655962492290296
Avg bias post: 0.15039805362099096
Ttest_1sampResult(statistic=0.3338079183941443, pvalue=0.7386

In [23]:
# Cross-fitting re-estimating the penalty parameters
print("Start time: {}".format(datetime.datetime.now().replace(microsecond=0)))
honest_predictions_simplex2, cf_fits_full_simplex2 = SC.get_c_predictions_honest(X_Y_pre[control_units,:], Y_post[control_units,:], Y[control_units,:], cf_folds=38, verbose=1, progress=False, fast=False, n_multi=n_multi, constrain="simplex", w_pen_inner=True)
print("End time: {}".format(datetime.datetime.now().replace(microsecond=0)))
full_simplex2_Y_sc_xf = full_simplex_Y_sc.copy()
full_simplex2_Y_sc_xf[control_units,:] = honest_predictions_simplex2
print_summary(full_simplex_fit, Y_pre, Y_post, full_simplex2_Y_sc_xf)

Start time: 2021-01-21 14:18:06
CROSS-FITTING:
 |>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>| 
End time: 2021-01-21 15:05:21
V>0: [0.04306114 0.04854367 0.05662882 0.05987692 0.0607689  0.06293688
 0.0692605  0.06679674 0.06189289 0.05663904 0.05521077 0.05218668
 0.05268511 0.04874349 0.04189329 0.04094062 0.0411164  0.03987489
 0.04094325]
#V>0: 1.0
['cigsale(1970)' 'cigsale(1971)' 'cigsale(1972)' 'cigsale(1973)'
 'cigsale(1974)' 'cigsale(1975)' 'cigsale(1976)' 'cigsale(1977)'
 'cigsale(1978)' 'cigsale(1979)' 'cigsale(1980)' 'cigsale(1981)'
 'cigsale(1982)' 'cigsale(1983)' 'cigsale(1984)' 'cigsale(1985)'
 'cigsale(1986)' 'cigsale(1987)' 'cigsale(1988)']
Avg bias pre: 0.001587579785291508
Ttest_1sampResult(statistic=0.03299404783990875, pvalue=0.9736884656718933)
Avg MSE pre: 1.6693055667696615
Avg max abs val pre:4.655921132940995
Avg bias post: 0.1503795071652061
Ttest_1sampResult(statistic=0.33376497828759094, pvalue=0.738710